In [74]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
from datetime import datetime
import re
import time
import streamlit as st

In [29]:
df = pd.read_csv('fake_jobs_scraped.csv')
df.rename(columns={'title': 'Job Title', 
                   'company': 'Company',
                    'location': 'Location',
                      'date': 'Date', 
                      'description':   'Description',
                      'apply_link': 'Apply Link'}, 
                      inplace=True)

pd.set_option('display.max_colwidth', 50)

# # تكبير عرض العمود Description
# df.style.set_table_styles([
#     {'selector': 'Description', 'props': [('min-width', '500px')]}  # اختار الرقم اللي يناسبك
# ])

df.head()

,Job Title,Company,Location,Date,Description,Apply Link,scrape_date,skill_python,skill_javascript,skill_django,skill_flask,skill_sql,skill_aws,skill_docker
0,senior python developer,"Payne, Roberts and Davis","Stewartbury, AA",2021-04-08,Learn how to build a Model Context Protocol (M...,https://www.realpython.com,2025-09-29T07:29:01.947902,True,False,False,False,False,False,False
1,energy engineer,Vasquez-Davidson,"Christopherville, AA",2021-04-08,Learn how to build a Model Context Protocol (M...,https://www.realpython.com,2025-09-29T07:29:01.947902,True,False,False,False,False,False,False
2,legal executive,"Jackson, Chambers and Levy","Port Ericaburgh, AA",2021-04-08,Learn how to build a Model Context Protocol (M...,https://www.realpython.com,2025-09-29T07:29:01.947902,True,False,False,False,False,False,False
3,fitness centre manager,Savage-Bradley,"East Seanview, AP",2021-04-08,Learn how to build a Model Context Protocol (M...,https://www.realpython.com,2025-09-29T07:29:01.947902,True,False,False,False,False,False,False
4,product manager,Ramirez Inc,"North Jamieview, AP",2021-04-08,Learn how to build a Model Context Protocol (M...,https://www.realpython.com,2025-09-29T07:29:01.947902,True,False,False,False,False,False,False


In [50]:
df.head()

df.to_csv("jops.csv", index=False)

In [ ]:

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

descriptions = []

for link in df['apply_link']:
    if pd.notna(link):
        page = requests.get(link, headers=headers)
        page.raise_for_status()
        time.sleep(1)
        soup = BeautifulSoup(page.text, 'lxml')
        desc_tag = soup.find("div", class_="description") or soup.find("p")
        if desc_tag:
            descriptions.append(desc_tag.get_text(" ", strip=True))
        else:
            descriptions.append("No description")
    else:
        descriptions.append("No link")

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

url = 'https://realpython.github.io/fake-jobs/'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}


page = requests.get(url, headers=headers)
page.raise_for_status()
soup = BeautifulSoup(page.text, 'lxml')

cards = soup.find_all('div', class_='card')

jobs = []

for card in cards:
    links = card.find_all('a', href=True)
    apply_link = links[1]['href'] if len(links) > 1 else None
    
    description = ""
    if apply_link:
        page_detail = requests.get(apply_link, headers=headers)
        page_detail.raise_for_status()
        time.sleep(1)  
        soup_detail = BeautifulSoup(page_detail.text, 'lxml')
        detail_desc_tag = soup_detail.find("div", class_="content")
        if detail_desc_tag:  
            description = detail_desc_tag.get_text(" ", strip=True)
    
    jobs.append({
        'title': card.find("h2").get_text(strip=True),
        'company': card.find("h3").get_text(strip=True),
        'location': card.find("p", class_="location").get_text(strip=True),
        'apply_link': apply_link,
        'description': description
    })


dd = pd.DataFrame(jobs)
print("Saved", len(df), "jobs -> jobs_with_descriptions.csv")




Saved 100 jobs -> jobs_with_descriptions.csv


In [ ]:
des = dd['description'].str.replace(r'[^a-zA-Z0-9\s,.]', '', regex=True)
df['Description'] = des
type(des)

pandas.core.series.Series

In [73]:
df.to_csv('jops.csv', index=False)

In [100]:
skills = ["skill_python","skill_javascript","skill_django","skill_flask","skill_sql","skill_aws","skill_docker"]


for skill in df["skill_sql"]:
    n = len(df)
    toggle_list = [False, True] * (n//2) 
    if n % 2 != 0:  
        toggle_list.append(True)
    df['skill_sql'] = toggle_list



In [110]:
df.columns = df.columns.str.strip()

df.columns.tolist()

df = df.loc[: , df.columns.notna()]

df.head()

df.to_csv('jops.csv', index=False)